<a href="https://colab.research.google.com/github/sh-0620/dacon-recommender-system/blob/main/vvs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict

In [4]:
# CSV 파일 로드
log_df = pd.read_csv("/content/drive/MyDrive/웹 기사 추천시스템/view_log.csv")  # 유저 로그 데이터
article__df = pd.read_csv("/content/drive/MyDrive/웹 기사 추천시스템/article_info.csv")  # 기사 데이터
article_nouns_df = pd.read_csv("/content/drive/MyDrive/웹 기사 추천시스템/article_nouns.csv")  # New기사 데이터

In [5]:
# 불필요한 열 제거
article_nouns_df = article_nouns_df.drop(['userCountry', 'userRegion', 'userID'], axis=1)

# 데이터 병합
merged_df = pd.merge(log_df, article_nouns_df, on='articleID')

# 기사 데이터 중 로그 데이터에 없는 기사 제거
article_nouns_df = article_nouns_df[article_nouns_df['articleID'].isin(merged_df['articleID'].unique())]

merged_df = pd.merge(log_df, article_nouns_df, on='articleID')

In [6]:
# 사용자-기사 행렬 생성
user_article_matrix = merged_df.groupby(['userID', 'articleID']).size().unstack(fill_value=0)

# 사용자 간의 유사성 계산
user_similarity = cosine_similarity(user_article_matrix.values)

# 협업 필터링 추천 점수 계산
user_based_scores = user_similarity.dot(user_article_matrix.values) / np.array([np.abs(user_similarity).sum(axis=1)]).T


In [7]:
# 라벨 인코더 생성 및 학습
article_label_encoder = LabelEncoder()
merged_df['articleID'] = article_label_encoder.fit_transform(merged_df['articleID'])

# 유저 아이디 라벨 인코딩 적용
user_label_encoder = LabelEncoder()
merged_df['userID'] = user_label_encoder.fit_transform(merged_df['userID'])


In [8]:
# merged_df 원핫 인코딩
encoder = OneHotEncoder()

# 범주형 변수 선택 (지역, 나라, 언어, 형식)
merged_df_categories = merged_df[['userRegion', 'userCountry', 'Format', 'Language']]


# One-Hot Encoding 수행
merged_df_encoded_categories = encoder.fit_transform(merged_df_categories).toarray()

In [10]:
# Content 컬럼의 길이 계산
merged_df['Content_length'] = merged_df['Content'].apply(len)

# Content 길이를 특징으로 추가
content_lengths = merged_df['Content_length'].values.reshape(-1, 1)

In [11]:
# 기사 데이터 전처리: 제목 + 본문
tfidf = TfidfVectorizer(
    stop_words='english',
    max_features=10000,
    ngram_range=(1, 2),
    max_df=0.8,
    min_df=3,
    sublinear_tf=True,
    smooth_idf=True
)
tfidf_matrix = tfidf.fit_transform(merged_df['Title'] + ' ' + merged_df['Content_Nouns'])

In [12]:
# TF-IDF 행렬과 원핫 인코딩된 피처 벡터 결합
combined_features = np.hstack((tfidf_matrix.toarray(), merged_df_encoded_categories, content_lengths ))

In [13]:
# 유저별 기사 읽기 패턴 추출
user_read_articles = defaultdict(list)
for _, row in merged_df.iterrows():
    user_read_articles[row['userID']].append(row['articleID'])

In [16]:
# 유저 기반 협업 필터링 모델
def recommend_articles(userID, top_n=5):

    if userID not in user_read_articles or not user_read_articles[userID]:
        return article_nouns_df.sample(n=top_n)[['Title', 'Content_Nouns']]

    read_articles = user_read_articles[userID]
    read_articles_features = combined_features[read_articles]

    if read_articles_features.shape[0] == 0:
        return article_nouns_df.sample(n=top_n)[['Title', 'Content_Nouns']]

    # 사용자가 읽은 기사들의 피처 벡터 평균 계산
    avg_features = np.mean(read_articles_features, axis=0).reshape(1, -1)

    # 사용자가 읽은 기사들의 피처 벡터 평균과 모든 기사들의 피처 벡터 간의 코사인 유사도 계산
    content_similarities = cosine_similarity(avg_features, combined_features).flatten()

    # 사용자 기반 협업 필터링 점수
    # user_idx = user_label_encoder.transform([userID])[0]
    collaborative_similarities = user_based_scores[userID]

    # 하이브리드 추천 점수 계산 (콘텐츠 기반 필터링과 협업 필터링의 가중합)
    # content_similarities와 collaborative_similarities의 크기를 맞추기 위해 content_similarities의 모든 기사에 대한 유사도를 계산
    hybrid_similarities = np.zeros(collaborative_similarities.shape)

    for i, article_id in enumerate(article_nouns_df['articleID']):
        if article_id in user_read_articles[userID]:
            idx = article_nouns_df[article_nouns_df['articleID'] == article_id].index[0]
            hybrid_similarities[i] = collaborative_similarities[i] + content_similarities[idx]
        else:
            hybrid_similarities[i] = collaborative_similarities[i]

    # 유사도가 높은 상위 top_n 개의 기사 인덱스 추출
    similar_articles_indices = hybrid_similarities.argsort()[-top_n:][::-1]

    # 추천 기사 반환
    recommendations = article_nouns_df.iloc[similar_articles_indices]
    return recommendations[['Title', 'Content_Nouns']]


In [17]:
recommend_articles(0)

,Title,Content_Nouns
411,Embracing Agile,idea problem method development function compa...
664,Running GV sprints inside corporates - learn f...,sprint corporates mistake venture sprint book ...
1568,Ray Kurzweil: The world isn't getting worse - ...,author inventor computer scientist futurist sp...
1230,You won't recognize the new world of digital p...,intelligence device device pocket task camera ...
2255,Equal Pay Day in the spotlight this year,woman money men woman cent dollar men income a...


In [ ]:
user_recommendations = []

for encoded_user_id in merged_df['userID'].unique():
    # 추천 기사 인덱스 가져오기
    recommended_articles = recommend_articles(encoded_user_id)

    # 원래 유저 아이디로 역변환
    original_user_id = user_label_encoder.inverse_transform([encoded_user_id])[0]

    for idx in recommended_articles.index:
        # 원래 기사 아이디로 역변환
        original_article_id = article_label_encoder.inverse_transform([merged_df.loc[idx, 'articleID']])[0]

        # 추천 결과를 리스트에 저장
        user_recommendations.append({
            'userID': original_user_id,
            'articleID': original_article_id
        })

In [ ]:
submission = pd.read_csv("/content/drive/MyDrive/웹 기사 추천시스템/sample_submission.csv")  # 유저 로그 데이터

In [ ]:
recommendations_df = pd.DataFrame(user_recommendations)

# sample_submission.csv 형태로 DataFrame 생성
submission = submission[['userID']].merge(recommendations_df, on='userID', how='left')
submission = submission[['userID', 'articleID']]

submission.to_csv('hybrid_recommendations_V1.csv', index=False)